In [1]:
# This cell should be exactly the same between all models

import pickle as pk

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from common import *
from util import *

# Load everything
scaled_data = {}
with np.load(PREPROCESSED_DATASET_FILEPATH) as npz_loader:
    for key in npz_loader:
        scaled_data[key] = npz_loader[key]
scaler = pk.load(open(PREPROCESSING_SCALER_FILEPATH, "rb"))

# Input and output dims
input_shape = tuple(list(scaled_data['x_train'].shape)[1:])
output_shape = tuple(list(scaled_data['y_train'].shape)[1:])
input_dims = np.product(input_shape)
output_dims = np.product(output_shape)

In [2]:
print("tensorflow: {}".format(tf.__version__))
print("numpy: {}".format(np.__version__))

tensorflow: 2.0.0
numpy: 1.17.0


In [3]:
# CONSTANTS:
MODEL_NAME = "Feedforward"

In [4]:
# Construct TF model here

# Simple Feedforward Neural Network
model = tf.keras.Sequential(
    [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(units=256,activation='relu',),
        #kernel_regularizer=tf.keras.regularizers.L2(0.01)),
        #tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=256,activation='relu',),
        #kernel_regularizer=tf.keras.regularizers.L2(0.01)),
        #tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(units=output_dims,activation='linear'),
        tf.keras.layers.Reshape(output_shape)
    ]
)

In [5]:
# hyperparameters and callbacks

BATCH_SIZE = 64
MAX_EPOCHS = 20

def scheduler(epoch, lr):
    if epoch<50:
        return lr
    elif epoch%20==0:
        return lr/10
    else:
        return lr
    
callback_lr = tf.keras.callbacks.LearningRateScheduler(scheduler)
callback_es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=10)

model.compile(loss='mean_absolute_error', optimizer=tf.keras.optimizers.Adam(1e-3))

In [6]:
# Compile and fit model here
history = model.fit(
    x=scaled_data['x_train'],
    y=scaled_data['y_train'],
    batch_size=BATCH_SIZE,
    epochs=MAX_EPOCHS,
    validation_data=(scaled_data['x_val'],
    scaled_data['y_val']),
    callbacks=[callback_lr,callback_es]
)

Train on 23974 samples, validate on 3425 samples
Epoch 1/20
23974/23974 [==============================] - 16s 664us/sample - loss: 0.2496 - val_loss: 0.2223
Epoch 2/20
23974/23974 [==============================] - 14s 578us/sample - loss: 0.2045 - val_loss: 0.2130
Epoch 3/20
23974/23974 [==============================] - 14s 575us/sample - loss: 0.1956 - val_loss: 0.2045
Epoch 4/20
23974/23974 [==============================] - 14s 573us/sample - loss: 0.1881 - val_loss: 0.2090
Epoch 5/20
23974/23974 [==============================] - 14s 568us/sample - loss: 0.1830 - val_loss: 0.1998
Epoch 6/20
23974/23974 [==============================] - 14s 565us/sample - loss: 0.1788 - val_loss: 0.1974
Epoch 7/20
23974/23974 [==============================] - 14s 576us/sample - loss: 0.1752 - val_loss: 0.1933
Epoch 8/20
23974/23974 [==============================] - 14s 576us/sample - loss: 0.1729 - val_loss: 0.1978
Epoch 9/20
23974/23974 [==============================] - 14s 574us/sample - lo

In [7]:
# Compute unnormalized prediction loss

preds = {}

for split in ['test','val']:
    preds[split] = model.predict(scaled_data['x_'+split])
    preds[split] = scaler.inverse_transform(preds[split])

In [8]:
# Save model
model.save(model_name_to_model_filepath(MODEL_NAME))

# Save run info
run_info = {}
run_info["history"] = history.history
run_info["predictions"] = preds # idk whether this part makes sense for RNNs or not
pk.dump(run_info, open(model_name_to_run_info_filepath(MODEL_NAME), "wb"))